In [673]:
import pandas as pd

ШАГ 1: ЗАГРУЗКА ДАННЫХ

In [674]:
workers = pd.read_csv("workers.csv")
equipment = pd.read_csv("equipment.csv")
schedule = pd.read_csv("schedule_template.csv")
requirements = pd.read_csv("position_requirements.csv")
plan = pd.read_csv("plan.csv")

print("workers - workers.csv")
display(workers.head())
print("equipment - equipment.csv")
display(equipment.head())
print("schedule - schedule_template.csv")
display(schedule.head())
print("requirements - position_requirements.csv")
display(requirements.head())
print("plan - plan.csv")
display(plan.head())

workers - workers.csv


,worker_id,name,плоская,высокая,струйная
0,W001,Работник П 01,7,6,5
1,W002,Работник П 02,6,5,0
2,W003,Работник П 03,5,4,0
3,W004,Работник П 04,4,0,5
4,W005,Работник П 05,7,6,0


equipment - equipment.csv


,machine_id,machine_type,operators_needed
0,PM-01,плоская,4
1,PM-02,плоская,4
2,PM-03,высокая,4
3,PM-04,высокая,4
4,SM-01,струйная,1


schedule - schedule_template.csv


,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


requirements - position_requirements.csv


,machine_type,position,min_rank,profession_required
0,плоская,1,7,Печатник
1,плоская,2,6,Печатник
2,плоская,3,5,Печатник
3,плоская,4,4,Печатник
4,струйная,1,5,Оператор струйной печати


plan - plan.csv


,machine_id,night,day,evening
0,PM-01,True,True,True
1,PM-02,True,True,True
2,PM-03,True,True,True
3,PM-04,True,True,True
4,SM-01,True,True,True


In [675]:
# Определяем основную профессию
cols = ["плоская", "высокая", "струйная"]
workers["основная_профессия"] = workers[cols].idxmax(axis=1)

# Добавляем все професии работника
workers["все_профессии"] = workers.apply(
    lambda row: [c for c in cols if row[c] > 0], axis=1
)

workers.head()

,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,W001,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,W002,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,W003,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,W004,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,W005,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [676]:
# Параметры: генерируем для недели 1, понедельника, дневной смены
target_week = 2
# target_day = "Понедельник"
target_shift = "День"
shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}

In [677]:
# Функция для переворота смены
def shift_next_week(week, shift_name):
    shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}
    print(f"shift_workers - Кандидаты для работы в смену: {shift_name} недели {week}")
    result = schedule[
        (schedule["week"] == week - 1)
        &
        # (schedule["day"] == target_day) &
        (schedule["shift"] == shift[shift_name])
    ]
    result = result.merge(workers, on="worker_id", how="left")
    print(f"В смене работает {len(result)} человек")

    return result

In [678]:
shift_workers_day = shift_next_week(target_week, "День")
display(shift_workers_day.head())

shift_workers_evning = shift_next_week(target_week, "Вечер")
# display(shift_workers_evning)

shift_workers_nigth = shift_next_week(target_week, "Ночь")
# display(shift_workers_nigth)

shift_workers - Кандидаты для работы в смену: День недели 2
В смене работает 20 человек


,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,1,Вечер,W021,Работник П 21,7,6,0,плоская,"[плоская, высокая]"
1,1,Вечер,W022,Работник П 22,6,5,0,плоская,"[плоская, высокая]"
2,1,Вечер,W023,Работник П 23,5,4,0,плоская,"[плоская, высокая]"
3,1,Вечер,W024,Работник П 24,4,0,5,струйная,"[плоская, струйная]"
4,1,Вечер,W025,Работник П 25,7,6,0,плоская,"[плоская, высокая]"


shift_workers - Кандидаты для работы в смену: Вечер недели 2
В смене работает 20 человек
shift_workers - Кандидаты для работы в смену: Ночь недели 2
В смене работает 20 человек


In [679]:
def build_shift_rotation(schedule: pd.DataFrame, workers: pd.DataFrame, target_week: int) -> pd.DataFrame:
    """
    Формирует общий датафрейм кандидатов на target_week для всех смен сразу,
    применяя правило переворота смен:
        Ночь -> День, День -> Вечер, Вечер -> Ночь
    """
    shift_map = {"Ночь": "Вечер", "День": "Ночь", "Вечер": "День"}

    # Базовый слой — прошлая неделя
    prev = schedule.loc[schedule["week"] == target_week - 1].copy()

    # Сохраним прошлую смену (на всякий случай для анализа)
    prev = prev.rename(columns={"shift": "prev_shift"})

    # Рассчитаем смену на следующую неделю и проставим неделю
    prev["shift"] = prev["prev_shift"].map(shift_map)
    prev["week"] = target_week

    # Объединим с данными по работникам
    result = prev.merge(workers, on="worker_id", how="left")

    # Короткая статистика по сменам
    counts = result["shift"].value_counts()
    total = int(counts.sum())
    print(f"Кандидаты на работу в неделю {target_week}: всего {total}")
    for s in ["День", "Вечер", "Ночь"]:
        print(f"  {s}: {int(counts.get(s, 0))}")

    return result


In [680]:
shift_workers_all = build_shift_rotation(schedule, workers, target_week)
display(shift_workers_all.head())

# # При необходимости — получить конкретную смену:
# shift_workers_day   = shift_workers_all.query('shift == "День"')
# shift_workers_even  = shift_workers_all.query('shift == "Вечер"')
# shift_workers_night = shift_workers_all.query('shift == "Ночь"')


Кандидаты на работу в неделю 2: всего 60
  День: 20
  Вечер: 20
  Ночь: 20


,week,prev_shift,worker_id,shift,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,2,Ночь,W001,Вечер,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,2,Ночь,W002,Вечер,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,2,Ночь,W003,Вечер,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,2,Ночь,W004,Вечер,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,2,Ночь,W005,Вечер,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [681]:
# Для целевой смены получаем оборудование и требования по работникам
def f_shift_equipment(plan, shif_name):
    shift_equipment = plan[plan["shift"] == shif_name][
        ["week", "shift", "machine_id", "machine_type"]
    ].merge(requirements, on="machine_type", how="left")
    shift_equipment["worker_id"] = None
    
    return shift_equipment

In [682]:
# Преобразуем в длинный формат
plan_long = plan.melt(
    id_vars="machine_id",
    value_vars=["night", "day", "evening"],
    var_name="shift",
    value_name="works",
)

# Оставляем только те строки, где машина работает
plan_long = (
    plan_long[plan_long["works"] == True].drop(columns="works").reset_index(drop=True)
)

# Заменяем названия смен на русские
plan_long = plan_long.replace(
    {"shift": {"night": "Ночь", "day": "День", "evening": "Вечер"}}
)

plan_long = plan_long.merge(
    equipment[["machine_id", "machine_type"]], on="machine_id", how="left"
)
plan_long["week"] = target_week

# Позже убрать дублирование кода
shift_equipment = f_shift_equipment(plan_long, target_shift)

# Создаем расписание для каждой смены с пустыми позициями работников
shift_equipment_day = f_shift_equipment(plan_long, 'День')
shift_equipment_evening = f_shift_equipment(plan_long, 'Вечер')
shift_equipment_night = f_shift_equipment(plan_long, 'Ночь')


display(shift_equipment_day.head())
display(shift_equipment_night.head())
display(shift_equipment_evening.head())

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,None
1,2,День,PM-01,плоская,2,6,Печатник,None
2,2,День,PM-01,плоская,3,5,Печатник,None
3,2,День,PM-01,плоская,4,4,Печатник,None
4,2,День,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Вечер,PM-01,плоская,1,7,Печатник,None
1,2,Вечер,PM-01,плоская,2,6,Печатник,None
2,2,Вечер,PM-01,плоская,3,5,Печатник,None
3,2,Вечер,PM-01,плоская,4,4,Печатник,None
4,2,Вечер,PM-02,плоская,1,7,Печатник,None


In [683]:
def candidat(s, profesion, min_rank, shift_name):
    # Базовая маска: работники в нужной смене и не назначенные ранее
    base_mask = (shift_workers_all["shift"] == shift_name) & (
        ~shift_workers_all["worker_id"].isin(appointed)
    )
    
    # Общая маска для "second" режима: профессия есть в списке всех профессий
    all_professions_mask = base_mask & (
        shift_workers_all["все_профессии"].apply(lambda x: profesion in x)
    )

    if s == "ferst":
        # Режим "ferst": ищем основной персонал с точным рангом
        candidates = shift_workers_all[
            base_mask
            & (shift_workers_all[profesion] == min_rank)
            & (shift_workers_all["основная_профессия"] == profesion)
        ]
    elif s == "second":
        # Режим "second": ищем персонал с точным min_rank ИЛИ min_rank + 1
        # Используем маску all_professions_mask для уменьшения дублирования
        rank_conditions = (shift_workers_all[profesion] == min_rank) | (
            shift_workers_all[profesion] == min_rank + 1
        )
        
        candidates = shift_workers_all[all_professions_mask & rank_conditions]
    elif s == "third":
        # Режим "third": ищем персонал c любым рангом
        # Используем маску all_professions_mask для уменьшения дублирования
        candidates = shift_workers_all[all_professions_mask]
    else:
        # Обработка неверного значения параметра 's'
        print(f"Ошибка: Неизвестный режим '{s}'. Используйте 'ferst' или 'second'.")
        candidates = pd.DataFrame() # Возвращаем пустой DataFrame

    return candidates


def position_assignment(shift_equipment, s="ferst", shift_name="День"):
    # appointed = set()
    free_machin_positions = []

    for index, row in shift_equipment.iterrows():
        if not row["worker_id"]:
            # print(row["worker_id"], "->", end=" ")
            profesion = row["machine_type"]
            min_rank = row["min_rank"]

            candidates = candidat(s, profesion, min_rank, shift_name)

            # candidates = shift_workers_day[
            #     (shift_workers_day["основная_профессия"] == profesion)
            #     & (shift_workers_day[profesion] == min_rank)
            #     & (~shift_workers_day["worker_id"].isin(appointed))
            # ]
            # print(candidates)

            if not candidates.empty:
                first = candidates.iloc[0]  # <-- первая строка
                worker_id = first["worker_id"]

                shift_equipment.loc[index, "worker_id"] = worker_id  # <-- loc, не iloc
                appointed.add(worker_id)
                # print(worker_id)
            else:
                # при необходимости логировать отсутствие кандидатов
                # print(f"Нет кандидатов для index={index}, {profesion}, min_rank={min_rank}")
                free_machin_positions.append(shift_equipment.loc[index])
        else:
            # print(row["worker_id"], "уже назначен")
            pass

    return pd.DataFrame(free_machin_positions), shift_equipment


appointed = set()

In [684]:
def run_assignment_for_shift(shift_equipment, default_tourse):
    # Первый тур назначения - основная профессия, минимальный ранг
    free_machin_positions, shift_equipment = position_assignment(shift_equipment, default_tourse[0][0], default_tourse[0][1])

    if not free_machin_positions.empty:
        print(f"Остались свободные позиции оборудования после тура 1 назначения:")
        for i in range(5):
            profession, shift_name = default_tourse[i + 1]
            # print(f"Остались свободные позиции оборудования после тура {i + 1} назначения:")
            # display(free_machin_positions)
            free_machin_positions, shift_equ = position_assignment(free_machin_positions, profession, shift_name)
            shift_equipment = shift_equipment.combine_first(shift_equ)
            if free_machin_positions.empty:
                break
            print(f"Остались свободные позиции оборудования после тура {i + 2} назначения:")
            # display(free_machin_positions)
    return shift_equipment

In [685]:
def summary_brigade(shift_equipment):
    # Считаем укомплектованность бригад
    summary = (
        shift_equipment
        .groupby(["machine_id", "machine_type"], as_index=False)
        .agg(
            required=("position", "count"),
            assigned=("worker_id", lambda s: s.notna().sum() - (s == "").sum())
        )
    )
    return summary

def incomplete_brigade(shift_equipment):
    # Определяем НЕПОЛНЫЕ БРИГАДЫ (частично укомплектованы)
    summary = summary_brigade(shift_equipment)
    incomplete = summary[
        (summary["assigned"] > 0) & (summary["assigned"] < summary["required"])
    ].copy()

    return incomplete

def decomlate_bregads(shift_equipment, appointed):
    # раскомплектовываем неполную бригаду
    incomplete = incomplete_brigade(shift_equipment)
    destaff = incomplete[incomplete['required'] / 2 >= incomplete['assigned']]['machine_id'].to_list()
    display(destaff)

    for m_id in destaff:
        mask = (shift_equipment['machine_id'] == m_id) & (shift_equipment['worker_id'].notna())
        appointed -= set(w_id for w_id in shift_equipment[mask]['worker_id'].to_list())
        # убираем назначенных работников
        shift_equipment.loc[mask, 'worker_id'] = None
    return appointed

In [686]:
def staff_brigades(shift_equipment, shift_name='Ночь', s='third'):
    incomplete = incomplete_brigade(shift_equipment)
    if incomplete.empty:
        return shift_equipment

    mask = shift_equipment["machine_id"].isin(incomplete["machine_id"])
    _, patch = position_assignment(shift_equipment.loc[mask].copy(), s=s, shift_name=shift_name)

    updated = shift_equipment.copy()
    # важно: не присваиваем результат .update(), она in-place
    updated.update(patch[["worker_id"]])
    return updated

# free_machin_positions, shift_equi = staff_brigades(shift_equipment_night)

# display(shift_equi)

# Собираем полное расписание смены обратно


In [687]:
# Назначение работников на позиции
default_tourse = [
    ("ferst",  "День"),
    ("second", "День"),
    ("ferst",  "Ночь"),
    ("second", "Ночь"),
    ("ferst",  "Вечер"),
    ("second", "Вечер"),
]

default_tourse_day = default_tourse.copy()
default_tourse_evening = default_tourse[4:] + default_tourse[:4]
default_tourse_night = default_tourse[2:] + default_tourse[:2]

shift_equipment_day = run_assignment_for_shift(shift_equipment_day, default_tourse_day)
# display(shift_equipment_day)
shift_equipment_evening = run_assignment_for_shift(shift_equipment_evening, default_tourse_evening)
# display(shift_equipment_evening)
shift_equipment_night = run_assignment_for_shift(shift_equipment_night, default_tourse_night)
# display(shift_equipment_night)

# shift_equipment_day

Остались свободные позиции оборудования после тура 1 назначения:
Остались свободные позиции оборудования после тура 2 назначения:
Остались свободные позиции оборудования после тура 3 назначения:
Остались свободные позиции оборудования после тура 4 назначения:
Остались свободные позиции оборудования после тура 5 назначения:
Остались свободные позиции оборудования после тура 1 назначения:
Остались свободные позиции оборудования после тура 2 назначения:
Остались свободные позиции оборудования после тура 3 назначения:
Остались свободные позиции оборудования после тура 4 назначения:
Остались свободные позиции оборудования после тура 5 назначения:
Остались свободные позиции оборудования после тура 6 назначения:
Остались свободные позиции оборудования после тура 1 назначения:
Остались свободные позиции оборудования после тура 2 назначения:
Остались свободные позиции оборудования после тура 3 назначения:
Остались свободные позиции оборудования после тура 4 назначения:
Остались свободные позици

In [688]:
# print(appointed)
print(len(appointed))

appointed = decomlate_bregads(shift_equipment_day, appointed)
appointed = decomlate_bregads(shift_equipment_evening, appointed)
appointed = decomlate_bregads(shift_equipment_night, appointed)



print(appointed)
print(len(appointed))
display(shift_equipment_night)

60


[]

[]

['PM-01']

{'W047', 'W028', 'W031', 'W046', 'W060', 'W021', 'W036', 'W059', 'W035', 'W018', 'W011', 'W048', 'W056', 'W037', 'W051', 'W039', 'W023', 'W027', 'W044', 'W010', 'W058', 'W015', 'W008', 'W042', 'W034', 'W055', 'W007', 'W001', 'W003', 'W017', 'W016', 'W014', 'W019', 'W033', 'W012', 'W057', 'W053', 'W054', 'W032', 'W038', 'W005', 'W004', 'W020', 'W041', 'W022', 'W006', 'W013', 'W002', 'W029', 'W025', 'W030', 'W024', 'W052', 'W050', 'W026', 'W049', 'W043', 'W040', 'W009'}
59


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
5,2,Ночь,PM-02,плоская,2,6,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
8,2,Ночь,PM-03,высокая,1,7,Печатник,W049
9,2,Ночь,PM-03,высокая,2,6,Печатник,W050


In [689]:
# Выводим свободные кандидаты после назначения
free_shift_workers = shift_workers_nigth[~shift_workers_nigth["worker_id"].isin(appointed)]
free_shift_workers

,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
4,1,День,W045,Работник П 45,7,6,0,плоская,"[плоская, высокая]"


In [690]:
# Назначение свободных кандидатов в свободные бригады
shift_equipment_day = staff_brigades(shift_equipment_day, 'Ночь')
shift_equipment_evening = staff_brigades(shift_equipment_evening, 'Ночь')
shift_equipment_night = staff_brigades(shift_equipment_night, 'Ночь')
display(shift_equipment_night)

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
5,2,Ночь,PM-02,плоская,2,6,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
8,2,Ночь,PM-03,высокая,1,7,Печатник,W049
9,2,Ночь,PM-03,высокая,2,6,Печатник,W050


In [691]:
# Выводим свободные кандидаты после назначения
free_shift_workers = shift_workers_nigth[~shift_workers_nigth["worker_id"].isin(appointed)]
free_shift_workers

,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии


In [668]:
# Собираем все смены в один график Ночь, День, Вечер
shift_equipment = pd.concat([shift_equipment_night, shift_equipment_day, shift_equipment_evening])

In [669]:
# Выводим результат с именами работников
result = shift_equipment.merge(
    workers[["worker_id", "name"]], on="worker_id", how="left"
)
result = result[result['worker_id'].notnull()]

In [670]:
# Группируем по машинам и позициям, чтобы получить минимальный разряд и требуемые профессии
by_machine_position = result.groupby(
    ["week", "shift", "machine_type", "machine_id", "position"], as_index=True
).agg(
    # machine_type=('machine_type', 'first'),
    # position = ('position', 'first'),
    min_rank=("min_rank", "min"),
    worker_id=("worker_id", "first"),
    name=("name", "first"),
    profession_required=("profession_required", "first"),
)

display(by_machine_position)

min_rank worker_id  \
week shift machine_type machine_id position                       
2    Вечер высокая      PM-03      1                7      W009   
                                   2                6      W010   
                                   3                5      W011   
                                   4                4      W042   
                        PM-04      1                7      W013   
                                   2                6      W014   
                                   3                5      W015   
                                   4                4      W046   
           плоская      PM-01      1                7      W005   
                                   2                6      W002   
                                   3                5      W003   
                                   4                4      W043   
                        PM-02      1                7      W041   
                                   2                6      W006   
                                   3                5      W007   
                                   4                4      W047   
     День  высокая      PM-03      1                7      W029   
                                   2                6      W030   
                                   3                5      W031   
                                   4                4      W039   
                        PM-04      1                7      W033   
                                   2                6      W034   
                                   3                5      W035   
                                   4                4      W040   
           плоская      PM-01      1                7      W021   
                                   2                6      W022   
                                   3                5      W023   
                                   4                4      W037   
                        PM-02      1                7      W025   
                                   2                6      W026   
                                   3                5      W027   
                                   4                4      W038   
           струйная     SM-01      1                5      W024   
                        SM-02      1                5      W028   
                        SM-03      1                5      W032   
                        SM-04      1                5      W036   
                        SM-05      1                5      W044   
                        SM-06      1                5      W048   
                        SM-07      1                5      W052   
                        SM-08      1                5      W056   
                        SM-09      1                5      W057   
                        SM-10      1                5      W058   
                        SM-11      1                5      W059   
                        SM-12      1                5      W060   
                        SM-13      1                5      W004   
                        SM-14      1                5      W008   
                        SM-15      1                5      W012   
                        SM-16      1                5      W016   
                        SM-17      1                5      W001   
                        SM-18      1                5      W017   
                        SM-19      1                5      W018   
                        SM-20      1                5      W019   
                        SM-21      1                5      W020   
     Ночь  высокая      PM-03      1                7      W049   
                                   2                6      W050   
                                   3                5      W051   
                                   4                4      W045   
                        PM-04      1                7      W053   
                         

In [671]:
schedule.head()

,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


In [672]:
temp = result[result['worker_id'].notnull()][["week", "shift", "worker_id"]]
print(temp)

    week  shift worker_id
8      2   Ночь      W049
9      2   Ночь      W050
10     2   Ночь      W051
11     2   Ночь      W045
12     2   Ночь      W053
13     2   Ночь      W054
14     2   Ночь      W055
20     2   День      W021
21     2   День      W022
22     2   День      W023
23     2   День      W037
24     2   День      W025
25     2   День      W026
26     2   День      W027
27     2   День      W038
28     2   День      W029
29     2   День      W030
30     2   День      W031
31     2   День      W039
32     2   День      W033
33     2   День      W034
34     2   День      W035
35     2   День      W040
36     2   День      W024
37     2   День      W028
38     2   День      W032
39     2   День      W036
40     2   День      W044
41     2   День      W048
42     2   День      W052
43     2   День      W056
44     2   День      W057
45     2   День      W058
46     2   День      W059
47     2   День      W060
48     2   День      W004
49     2   День      W008
50     2   Д